In [1]:
import torch
from GCN.data_processing import Data_Loader
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False
import numpy as np
import io
import os
import time
import json
import torch
import zipfile
import numpy as np
import torch.nn as nn
from PIL import Image,ImageOps
import torch.nn.functional as F
from vidaug import augmentors as va
from einops import rearrange, repeat
import math
from torch import einsum
from argparse import ArgumentParser
from core.models.curvenet_cls import CurveNet
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import argparse
random_seed = 420 #for reproducibility
from math import sqrt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error, mean_absolute_error
from torchsummary import summary
import colorama
from colorama import Fore, Back, Style
colorama.init(autoreset=True)
np.seterr(invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
!nvidia-smi

Thu Jan 19 19:31:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 526.86       Driver Version: 526.86       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   68C    P8     4W /  N/A |      0MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import numpy as np
from IPython.core.debugger import set_trace

class Graph():
    def __init__(self, num_node):
        self.num_node = num_node
        self.AD, self.AD2, self.bias_mat_1, self.bias_mat_2 = self.normalize_adjacency()
        
    def normalize_adjacency(self):
        self_link = [(i, i) for i in range(self.num_node)]
#         print('self', self_link)
        neighbor_1base = [(1, 2), (2, 21), (3, 21), (4, 3), (5, 21),
                                      (6, 5), (7, 6), (8, 7), (9, 21), (10, 9),
                                      (11, 10), (12, 11), (13, 1), (14, 13), (15, 14),
                                      (16, 15), (17, 1), (18, 17), (19, 18), (20, 19),
                                      (22, 23), (23, 8), (24, 25), (25, 12)]
        neighbor_link = [(i - 1, j - 1) for (i, j) in neighbor_1base]
#         print('neighbour: ', neighbor_link)
        edge = self_link + neighbor_link
#         print('Edge: ' ,edge)
        A = np.zeros((self.num_node, self.num_node)) # adjacency matrix
        print(A.shape)
        for i, j in edge:
            A[j, i] = 1
            A[i, j] = 1
        print(A)
        A2 = np.zeros((self.num_node, self.num_node)) # second order adjacency matrix
        for root in range(A.shape[1]):
            for neighbour in range(A.shape[0]):
                if A[root, neighbour] == 1:
                    for neighbour_of_neigbour in range(A.shape[0]):
                        if A[neighbour, neighbour_of_neigbour] == 1:
                            A2[root,neighbour_of_neigbour] = 1  
#         print(A2)
        #AD = self.normalize(A)
        #AD2 = self.normalize(A2)
        bias_mat_1 = np.zeros(A.shape)
        bias_mat_2 = np.zeros(A2.shape)
        bias_mat_1 = np.where(A!=0, bias_mat_1, -1e9)
        bias_mat_2 = np.where(A2!=0, A2, -1e9)
        AD = A.astype('float32')
        AD2 = A2.astype('float32')
        bias_mat_1 = bias_mat_1.astype('float32')
        bias_mat_2 = bias_mat_2.astype('float32')
        AD = torch.tensor(AD)
        AD2= torch.tensor(AD2)
        bias_mat_1 = torch.tensor(bias_mat_1)
        bias_mat_2 = torch.tensor(bias_mat_2)
        return AD, AD2, bias_mat_1, bias_mat_2
        
    def normalize(self, adjacency):
        rowsum = np.array(adjacency.sum(1))
        r_inv = np.power(rowsum, -1).flatten()
        r_inv[np.isinf(r_inv)] = 0
        r_mat_inv = np.diag(r_inv)
        normalize_adj = r_mat_inv.dot(adjacency)
        normalize_adj = normalize_adj.astype('float32')
        normalize_adj = torch.tensor(normalize_adj)   
        return normalize_adj

In [4]:
# print(len(np.unique(train_y)))
# print(np.unique(train_y))

In [5]:
# print(len(train_x))
# print(len(train_y))
# print(len(test_x))
# print(len(test_y))

In [6]:
# train_x.shape

In [7]:
# train_y[0].shape

In [8]:
# x = torch.tensor(train_x)
# x.size()

In [9]:
class Embedder:
    def __init__(self, **kwargs):
        self.kwargs = kwargs
        self.create_embedding_fn()

    def create_embedding_fn(self):
        embed_fns = []
        d = self.kwargs['input_dims']
        out_dim = 0
        if self.kwargs['include_input']:
            embed_fns.append(lambda x : x)
            out_dim += d

        max_freq = self.kwargs['max_freq_log2']
        N_freqs = self.kwargs['num_freqs']

        if self.kwargs['log_sampling']:
            freq_bands = 2.**torch.linspace(0., max_freq, steps=N_freqs)
        else:
            freq_bands = torch.linspace(2.**0., 2.**max_freq, steps=N_freqs)

        for freq in freq_bands:
            for p_fn in self.kwargs['periodic_fns']:
                embed_fns.append(lambda x, p_fn=p_fn, freq=freq : p_fn(x * freq))
                out_dim += d

        self.embed_fns = embed_fns
        self.out_dim = out_dim

    def embed(self, inputs):
        normalized = torch.cat([fn(inputs) for fn in self.embed_fns], -1)
        return normalized
    
def get_embedder(multires = 10, i=0):
    if i == -1:
        return nn.Identity(), 1

    embed_kwargs = {
                'include_input' : True,
                'input_dims' : 1,
                'max_freq_log2' : multires-1,
                'num_freqs' : multires,
                'log_sampling' : True,
                'periodic_fns' : [torch.sin, torch.cos],
    }

    embedder_obj = Embedder(**embed_kwargs)
    embed = lambda x, eo=embedder_obj : eo.embed(x)
    return embed, embedder_obj.out_dim

embeder = get_embedder()[0]    


In [10]:
class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn

    def forward(self, x, **kwargs):
        return self.fn(self.norm(x), **kwargs)


# GELU -> Gaussian Error Linear Units
class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout = 0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)


class RemixerBlock(nn.Module):
    def __init__(
        self,
        dim,
        seq_len,
        causal = False,
        bias = False
    ):
        super().__init__()
        self.causal = causal
        self.proj_in = nn.Linear(dim, 2 * dim, bias = bias)
        self.mixer = nn.Parameter(torch.randn(seq_len, seq_len))
        self.alpha = nn.Parameter(torch.tensor(0.))
        self.proj_out = nn.Linear(dim, dim, bias = bias)

    def forward(self, x):
        mixer, causal, device = self.mixer, self.causal, x.device
        x, gate = self.proj_in(x).chunk(2, dim = -1)
        x = F.gelu(gate) * x

        if self.causal:
            seq = x.shape[1]
            mask_value = -torch.finfo(x.dtype).max
            mask = torch.ones((seq, seq), device = device, dtype=torch.bool).triu(1)
            mixer = mixer[:seq, :seq]
            mixer = mixer.masked_fill(mask, mask_value)

        mixer = mixer.softmax(dim = -1)
        mixed = einsum('b n d, m n -> b m d', x, mixer)

        alpha = self.alpha.sigmoid()
        out = (x * mixed) * alpha + (x - mixed) * (1 - alpha)

        return self.proj_out(out)


class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x: torch.FloatTensor) -> torch.FloatTensor:
        """
        Args:
            x: `embeddings`, shape (batch, max_len, d_model)
        Returns:
            `encoder input`, shape (batch, max_len, d_model)
        """
        x = x + self.pe[:, : x.size(1)]
        return self.dropout(x)


class Attention(nn.Module):
    def __init__(self, dim, heads = 8, dim_head = 64, dropout = 0.):
        super().__init__()
        
        # print(f'Attention:: {dim} - {heads} - {dim_head} - {dropout}')

        inner_dim = dim_head *  heads
        project_out = not (heads == 1 and dim_head == dim)

        self.heads = heads
        self.scale = dim_head ** -0.5

        self.pos_embedding = PositionalEncoding(dim,0.1,128)
        self.attend = nn.Softmax(dim = -1)
        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)

        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        ) if project_out else nn.Identity()

    def forward(self, x):
        x += self.pos_embedding(x)
        qkv = self.to_qkv(x).chunk(3, dim = -1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = self.heads), qkv)

        dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale

        attn = self.attend(dots)

        out = torch.matmul(attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        return self.to_out(out)


class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout = 0.):
        super().__init__()

        # print('\n')
        # print(f'Transformers:: {dim} - {depth} - {heads} - {dim_head} - {mlp_dim}')

        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                PreNorm(dim, Attention(dim, heads = heads, dim_head = dim_head, dropout = dropout)),
                PreNorm(dim, FeedForward(dim, mlp_dim, dropout = dropout))
                #PreNorm(dim, RemixerBlock(dim,17))
            ]))

    def forward(self, x, swap = False):
        if swap: # for the self.transformer(x,swap = True)
            b, t, n , c = x.size() 
        for idx, (attn, ff) in enumerate(self.layers):
            if swap: # for the self.transformer(x,swap = True)
                if idx % 2 == 0:
                    #* attention along with all timesteps(frames) for each point(landmark)
                    x = rearrange(x, "b t n c -> (b n) t c")
                else:
                    #* attention to all points(landmarks) in each timestep(frame)
                    x = rearrange(x, "b t n c -> (b t) n c")
            x = attn(x) + x  # skip connections
            x = ff(x) + x    # skip connections
            
            # Now return the input x to its original formation
            if swap: # for the self.transformer(x,swap = True)
                if idx % 2 == 0:
                    x = rearrange(x, "(b n) t c -> b t n c", b = b)
                else:
                    x = rearrange(x, "(b t) n c -> b t n c", b = b)
                
        return x


class TemporalModel(nn.Module):
    
    def __init__(self):
        super(TemporalModel,self).__init__()
                
        self.encoder  =  CurveNet() # curve aggregation, needed for Point Clouds Shape Analysis. 
        self.downsample = nn.Sequential(
                            nn.Conv1d(120, 32, kernel_size=1, bias=False),
                            nn.BatchNorm1d(32),
                            # nn.Dropout(p=0.25), #* NEW
                            #nn.ReLU(inplace=True),
                            #nn.Conv1d(128, 32, kernel_size=1, bias=False),
                            #nn.BatchNorm1d(32),
                            )
        
        self.transformer = Transformer(256, 6, 4, 256//4, 256 * 2, 0.1)
        self.time = Transformer(256, 3, 4, 256//4, 256 * 2, 0.1)
        self.dropout = nn.Dropout(0.1)
        self.mlp_head = nn.Sequential(
            nn.LayerNorm(256),
            nn.Linear(256,1),
#             nn.Sigmoid()
        )
        
    def forward(self,x):
        b,t,n,c = x.size()
    
        x = rearrange(x, "b t n c -> (b t) c n")
        x = rearrange(self.dropout(self.encoder(x)), "b c n -> b n c") 
        x = self.downsample(x).view(b,t,32,-1) #b t 32 c
        x = self.transformer(x,swap = True).view(b,t,-1,256).mean(2)
        x = self.time(x).mean(1)
        x = self.mlp_head(x)
        return x

min_xyz = np.array([0.06372425, 0.05751023, -0.08976112]).reshape(1,1,3)
max_xyz = np.array([0.63246971, 1.01475966, 0.14436169]).reshape(1,1,3)


In [11]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
def performance_metrics(test_y, y_pred):
    test_dev = abs(test_y-y_pred)
    mean_abs_dev = torch.mean(test_dev)
    mae = mean_absolute_error(test_y, y_pred)
    rms_dev = sqrt(mean_squared_error(y_pred, test_y))
    mse = mean_squared_error(test_y,y_pred) 
    mape = mean_absolute_percentage_error(test_y, y_pred)
    return mae, rms_dev, mse, mape

In [12]:
def train(epochs,training_generator,test_generator,file):
    
    con = []      
    net = TemporalModel()
    net.cuda()
    
    lr = 0.0005
    optimizer = torch.optim.AdamW(net.parameters(), lr=lr,weight_decay= 0.0)
    lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[299], gamma=0.1)
    loss_func = torch.nn.HuberLoss(reduction='mean', delta= 0.1)
    start_time = time.time()
    best_accuracy = 500

    for epoch in range(epochs):
        pred_label = []
        true_label = []
        number_batch = 0
        for x, y in tqdm(training_generator, desc=f"Epoch {epoch}/{epochs-1}", ncols=60):
            if torch.cuda.device_count() > 0:
                x = x.cuda()
                y = y.cuda()
            x = torch.unsqueeze(x, 1) 
#             print(x.size())
            b,d,t,n,c = x.size()
            x = x.view(-1,t,n,c)
            pred = net(x)
            loss = loss_func(pred,y)
            
            pred_label.append(pred)
            true_label.append(y)
            
                        
            optimizer.zero_grad()
            loss.backward()

            optimizer.step()
            number_batch += 1
            lr = lr_scheduler.get_last_lr()[0]

        lr_scheduler.step()
        
        pred_label = torch.tensor(pred_label).flatten()
        true_label = torch.tensor(true_label).flatten()
#         print(true_label)
#         print(pred_label)
#         pred_label = torch.cat(pred_label,0)
#         true_label = torch.cat(true_label,0)
#         train_accuracy = torch.sum(pred_label == true_label).type(torch.FloatTensor) / true_label.size(0)

        mae, rms_dev, mse, mape = performance_metrics(true_label, pred_label)
    
        output('Epoch: ' + 'train' + str(epoch) + 
              '| mae ' + str(mae) + 
              '| rms_dev: ' + str(rms_dev) +
              '| mse: ' + str(mse) +
                '| mape: ' + str(mape)
              )
        print('Epoch: ' + 'train' + str(epoch) + 
              '| mae ' + str(mae) + 
              '| rms_dev: ' + str(rms_dev) +
              '| mse: ' + str(mse) +
              '| mape: ' + str(mape)
              )
        net.eval()
        pred_label = []
        pred_avg   = []
        true_label = []
        with torch.no_grad():
          for x, y in tqdm(test_generator, desc=f"Epoch {epoch}/{epochs-1}", ncols=60):

              if torch.cuda.device_count() > 0:
                  x = x.cuda()
                  y = y.cuda()
              x = torch.unsqueeze(x, 1)
              b,d,t,n,c = x.size()
              x = x.view(-1,t,n,c)
              pred_y    = net(x)
            ###commented to convert round
#               pred_mean = (pred_y.view(b,d).mean(1,keepdim = True) >= 0.5).float().cpu().detach()
#               pred_y    = ((pred_y).view(b,d).mean(1,keepdim = True) >= 0.5).float().cpu().detach()
              pred_label.append(pred_y)
#               pred_avg.append(pred_mean)
              true_label.append(y.cpu())
              
#           pred_label = torch.cat(pred_label,0)
# #           pred_avg   = torch.cat(pred_avg,0)  
#           true_label = torch.cat(true_label,0)
          
          pred_label = torch.tensor(pred_label).flatten()
          true_label = torch.tensor(true_label).flatten()
#           print(true_label.size())
#           print(pred_label.size())
          mae, rms_dev, mse, mape = performance_metrics(true_label, pred_label)
#           test_accuracy = torch.sum(pred_label == true_label).type(torch.FloatTensor) / true_label.size(0)
#           test_avg      = torch.sum(pred_avg   == true_label).type(torch.FloatTensor) / true_label.size(0)
          con.append([epoch,mae])
          output('test accuracy: ' + 
              '| mae ' + str(mae) + 
              '| rms_dev: ' + str(rms_dev) +
              '| mse: ' + str(mse) +
              '| mape: ' + str(mape)
              )
          print(Fore.GREEN + 'test accuracy: ' + 
              '| mae ' + str(mae) + 
              '| rms_dev: ' + str(rms_dev) +
              '| mse: ' + str(mse) +
              '| mape: ' + str(mape)
              )

          if mae < best_accuracy:
              filepath = f"{file}-{epoch:}-{loss}-{mae}.pt"
              torch.save(net.state_dict(), filepath)
            #   torch.save(net, filepath)
            #   test_frames(f'{test_accuracy}={test_f}')
              print("Better Results Achieved!!!!!!!!!!!!")
              best_accuracy = mae

        net.train()
        
        output(f"ETA Per Epoch:{(time.time() - start_time) / (epoch + 1)}")
        # print(f"ETA Per Epoch:{(time.time() - start_time) / (epoch + 1)}")

    best_v = max(con,key = lambda x:x[1])
    global perf
    perf += f"best accruacy is {best_v[1]} in epoch {best_v[0]}" + "\n"
    output(perf)
    
    
# image_size = 48
# label_path = "labels"
# data = "npy"

# sometimes = lambda aug: va.Sometimes(0.5, aug)
# seq = va.Sequential([
#     va.RandomCrop(size=(image_size, image_size)),       
#     sometimes(va.HorizontalFlip()),              
# ])


# label_path = "labels"


In [13]:
def main():
    
    # Add the arguments
    ####################Parser not working###########################
    ex = "Kimore_ex5"
    lr = 0.0001
    epoch = 1000
    batch_size = 10
    ####################Parser not working###########################

    # my_parser.add_argument('--ex', type=str, default="Kimore_ex5",
    #                        help='the name of exercise.', required=True)
    # my_parser.add_argument('--lr', type=int, default= 0.0001,
    #                        help='initial learning rate for optimizer.')
    # my_parser.add_argument('--epoch', type=int, default= 1000,
    #                        help='number of epochs to train.')
    # my_parser.add_argument('--batch_size', type=int, default= 10,
    #                        help='training batch size.')
    # args = my_parser.parse_args()
    data_loader = Data_Loader(ex) 

    train_x, test_x, train_y, test_y = train_test_split(data_loader.scaled_x, data_loader.scaled_y, test_size=0.2, 
                                                        random_state = random_seed)
    global output
    def output(s):
        with open(f"log_{ex}","a") as f:
            f.write(str(s) + "\n")
    print(train_x.shape)
    print(train_y.shape)

    train_x = torch.Tensor(train_x)
    train_y = torch.Tensor(train_y)
    test_x  = torch.Tensor(test_x)
    test_y = torch.Tensor(test_y)
    dataset = torch.utils.data.TensorDataset(train_x, train_y)
    training_generator = torch.utils.data.DataLoader(dataset, batch_size = 1, shuffle = True, num_workers = 2)
    dataset = torch.utils.data.TensorDataset(test_x, test_y)
    test_generator = torch.utils.data.DataLoader(dataset, batch_size = 1, shuffle = False, num_workers = 1)
    current_path = './models/'
    train(epoch,training_generator,test_generator,current_path)


In [ ]:
if __name__ == "__main__":
    main()

(298, 100, 25, 3)
(298, 1)


Epoch 0/999: 100%|████████| 298/298 [03:19<00:00,  1.50it/s]


Epoch: train0| mae 0.9427336| rms_dev: 1.1870208576686962| mse: 1.4090185| mape: 372.0898628234863


Epoch 0/999: 100%|██████████| 75/75 [00:28<00:00,  2.67it/s]


test accuracy: | mae 1.0014786| rms_dev: 1.215198820667561| mse: 1.4767082| mape: 225.1978874206543
Better Results Achieved!!!!!!!!!!!!


...

Epoch 9/999: 100%|████████| 298/298 [05:14<00:00,  1.06s/it]


Epoch: train9| mae 0.7973256| rms_dev: 0.9975021279104381| mse: 0.9950105| mape: 156.5334916114807


Epoch 9/999: 100%|██████████| 75/75 [00:34<00:00,  2.20it/s]


test accuracy: | mae 0.8565474| rms_dev: 1.0441031127082425| mse: 1.0901513| mape: 127.14672088623047


Epoch 10/999:  36%|██▍    | 106/298 [01:54<03:07,  1.02it/s]

In [ ]:
idx = torch.zeros(100, 25, 21)
idx.size()

In [ ]:
idx[:,:,1:].size()

In [ ]:
TypeError: mean() received an invalid combination of arguments - got (axis=NoneType, dtype=NoneType, out=NoneType, ), but expected one of:
 * (*, torch.dtype dtype)
 * (tuple of ints dim, bool keepdim, *, torch.dtype dtype)
 * (tuple of names dim, bool keepdim, *, torch.dtype dtype)


In [ ]:
y_true = torch.tensor([1,2,3,4,5,6], dtype = torch.float32)
y_pred = torch.tensor([1.5,2.5,3.5,4.5,5.5,6.5])

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
def performance_metrics(test_y, y_pred):
    test_dev = abs(test_y-y_pred)
#     print(test_dev)
    mean_abs_dev = torch.mean(test_dev)
    print(mean_abs_dev)
    mae = mean_absolute_error(test_y, y_pred)
    rms_dev = sqrt(mean_squared_error(y_pred, test_y))
    mse = mean_squared_error(test_y,y_pred) 
    mape = mean_absolute_percentage_error(test_y, y_pred)
    return mae, rms_dev, mse, mape

In [ ]:
performance_metrics(y_true, y_pred)